In [ ]:
# Import and initialize
import datetime
import numpy
import pygetm

%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.domain.create_cartesian(500.*numpy.arange(131), 500.*numpy.arange(2), 20, lat=0, H=20, interfaces=True)
sim = pygetm.Simulation(domain,
                        runtype=pygetm.BAROCLINIC, 
                        advection_scheme=pygetm.AdvectionScheme.HSIMT,
                        airsea=pygetm.airsea.Fluxes(),
                        log_level='ERROR',
                        internal_pressure_method=0
)

#KB set mask all around the basin

sim.temp.fill(20.)
sim.salt.fill(numpy.where(domain.T.x.all_values<32500., 20, 30))

# Idealized surface forcing
sim.airsea.taux.set(0.1)
sim.airsea.tauy.set(0.0)
sim.airsea.sp.set(0.0)
sim.airsea.shf.set(0.0)
sim.radiation.A.set(0.)
sim.radiation.kc1.set(0.)
sim.radiation.kc2.set(0.)

nsample_x = 2
nsample_z = 2
iy = 0
fig, ax = matplotlib.pyplot.subplots()
u = sim.pk.interp(domain.T) / domain.T.H
w = sim.ww.interp(domain.T, z=pygetm.CENTERS)
z_2d = domain.T.zc[::nsample_z, iy, ::nsample_x]
x_2d = numpy.broadcast_to(domain.T.x[iy, ::nsample_x], z_2d.shape)
pc = ax.pcolormesh(domain.T.xi[iy, :], domain.T.zf[:, iy, 0], sim.salt.values[:, iy, :])
Q = ax.quiver(x_2d, z_2d, u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x], scale=.01)
title = ax.set_title('time: 0 s')


In [ ]:
timestep = 15.
ntime = 24*int(3600. // timestep)
times = timestep * numpy.arange(ntime)
plotting_interval = 5

sim.start(datetime.datetime(2015, 1, 1), timestep=timestep, split_factor=20, report=180)
for istep, time in enumerate(times):
    sim.advance()
    if istep % plotting_interval == 0:
        u = sim.pk.interp(domain.T) / domain.T.H
        sim.ww.interp(w)
        Q.set_UVC(u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x])
        pc.set_array(sim.salt.values[:, iy, :].ravel())
        title.set_text('time: %s s' % time)
        fig.canvas.draw()

sim.finish()